# setup

In [1]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [2]:
import torch
import torch.nn as nn

from models.baseline import ResnetBaseline
from runners.multiclass import Runner

In [3]:
model_label = 'code15'
epochs = 10

from dataloaders.code15 import CODE as DS
from dataloaders.code15 import CODEsplit as DSsplit

# init

In [4]:
database = DS()
model = ResnetBaseline(n_classes = 6)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
runner = Runner(device = device, model = model, database = database, split = DSsplit, model_label = model_label)

# run

In [6]:
runner.train(epochs)

  0%|          | 0/2512 [00:00<?, ?it/s]

-- epoch 0


  4%|▍         | 109/2512 [02:46<1:00:59,  1.52s/it]


In [ ]:
runner.eval()